In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import roc_auc_score
import xgboost as xgb
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
import mlflow
import mlflow.sklearn

# 데이터 로딩
data = pd.read_csv('../data/dataset.csv')

# Tracking Server 설정
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000") # 환경 변수를 설정했지만, 확실하게 설정

In [2]:
# 예측 타겟과 특성 분리
X = data.drop('Default', axis=1)
y = data['Default']

In [3]:
# 범주형과 수치형 컬럼 구분
categorical_cols = ['State', 'BankState', 'NewExist', 'UrbanRural', 'RealEstate']
numerical_cols = ['DisbursementGross', 'GrAppv', 'daysterm']


In [4]:
# 데이터 전처리 파이프라인 구성
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numerical_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
])

In [5]:
# 데이터 전처리 실행
X_processed = preprocessor.fit_transform(X)

In [6]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

# 하이퍼파라미터 탐색 공간 설정
space = {
    'max_depth': hp.choice('max_depth', range(3, 10)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'n_estimators': hp.choice('n_estimators', range(50, 200)),
    'gamma': hp.uniform('gamma', 0, 5)
}


In [9]:
# 최적화를 위한 목적 함수 정의
def objective(params):
    # MLflow에 실험 이름 설정
    mlflow.set_experiment("assignment1")

    # 각 하이퍼파라미터 조합별 실험 시작
    with mlflow.start_run(nested=True):
        # XGBoost 모델 초기화 및 훈련
        model = xgb.XGBClassifier(eval_metric='logloss', use_label_encoder=False, **params)
        model.fit(X_train, y_train)

        # 예측 확률 및 ROC-AUC 계산
        probs = model.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, probs)

        # MLflow에 파라미터 및 메트릭 로깅
        mlflow.log_params(params)
        mlflow.log_metric("roc_auc", auc)

        # 목적 함수 결과 반환 (손실 최소화)
        return {'loss': -auc, 'status': STATUS_OK}


In [10]:
# Hyperopt를 사용한 최적화 실행
trials = Trials()
best_params = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=30,
    trials=trials
)

  0%|          | 0/30 [00:00<?, ?trial/s, best loss=?]

c:\Users\SSAFY\venvs\mlflow-env\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:15:05] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



🏃 View run upset-bear-623 at: http://127.0.0.1:5000/#/experiments/1/runs/d05b938601014c44bb19d75cfc79b031

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1

🏃 View run resilient-goose-236 at: http://127.0.0.1:5000/#/experiments/1/runs/850498acafd74b51927f4d6a5d7a6fdd

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                     

  7%|▋         | 2/30 [00:00<00:07,  3.77trial/s, best loss: -0.9784674784674785]

c:\Users\SSAFY\venvs\mlflow-env\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:15:05] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



🏃 View run amusing-cod-648 at: http://127.0.0.1:5000/#/experiments/1/runs/d5e83e546a0d48d8a4f0da61cfdd46e9

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                     

 10%|█         | 3/30 [00:00<00:06,  4.21trial/s, best loss: -0.9784674784674785]

...

c:\Users\SSAFY\venvs\mlflow-env\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:15:11] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



🏃 View run fearless-snail-270 at: http://127.0.0.1:5000/#/experiments/1/runs/b502811404ae4d348278e6f206bf5a3e

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                      

 97%|█████████▋| 29/30 [00:06<00:00,  4.30trial/s, best loss: -0.9803361053361053]

c:\Users\SSAFY\venvs\mlflow-env\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:15:11] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



🏃 View run nervous-toad-355 at: http://127.0.0.1:5000/#/experiments/1/runs/6190e810409c4c6aa8dd64ed9d0cedd9

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                      

100%|██████████| 30/30 [00:06<00:00,  4.32trial/s, best loss: -0.9803361053361053]


c:\Users\SSAFY\venvs\mlflow-env\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:15:12] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)

